Exercise 1



Ex 1.1

In [ ]:
%%bash
rm -fv ipopt*
apt install -q -yy libgfortran5
sleep 1
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/ipopt
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/test.nl
chmod u+rx ipopt

removed 'ipopt'
Reading package lists...
Building dependency tree...
Reading state information...
libgfortran5 is already the newest version (8.4.0-1ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


The above code also fetches an optimization problem in an '.nl' format. 'nl' format is suitable for solvers, not for humans. Lets call ipopt directly (without using any Python) to solve it.

In [ ]:
!./ipopt test.nl



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      230

Total number of variables............................:       30
                     variables with only lower bounds:       30
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ


If you are able to see Ipopt output, we can install pyomo as usual

In [ ]:
!pip install -q pyomo

In [ ]:
#Importing the required packages
import numpy as np
from pyomo.environ import *

Let us try to solve a nonlinear optimization problem without any constraints
$\min \ 100(x_2 - x_1^2)^2 + (1-x_1)^2$. Pyomo uses two asterisks (**) to denote 'raise to power'. Other mathematical operators like sqrt (for square root), log, sin, cos, multiplication and division etc are available.

In [ ]:
#Creating Model
model_ex1 = ConcreteModel()

In [ ]:
#Variable declaration
model_ex1.x1 = Var()
model_ex1.x2 = Var()

In [ ]:
#Objective function
model_ex1.obj = Objective(expr = ((1-model_ex1.x1)**2) + (((model_ex1.x2-model_ex1.x1**2)*100)**2) , sense=minimize)

In [ ]:
#Printing the details of model_ex1 that we have created
model_ex1.pprint() 

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : (1 - x1)**2 + ((x2 - x1**2)*100)**2

3 Declarations: x1 x2 obj


Ex 1.2

In [ ]:
lower_bound = 5
model_ex1.x1.setlb(lower_bound)
model_ex1.x2.setlb(lower_bound)

In [ ]:
upper_bound = 10
model_ex1.x1.setub(upper_bound)
model_ex1.x2.setub(upper_bound)

In [ ]:
#Printing the details of model_ex1 that we have created
model_ex1.pprint()

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     5 :  None :    10 : False :  True :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     5 :  None :    10 : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : (1 - x1)**2 + ((x2 - x1**2)*100)**2

3 Declarations: x1 x2 obj


Now loading the solver by passing the model to it and getting required result after solving 

In [ ]:
opt_ipopt = SolverFactory('ipopt')
result_ex1 = opt_ipopt.solve(model_ex1)
print(result_ex1)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.020053386688232422
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [ ]:
print('The Solver status:', result_ex1.solver.status)
print('The Solver termination condition:',result_ex1.solver.termination_condition)
print('Objective function value is = ', model_ex1.obj())

The Solver status: ok
The Solver termination condition: optimal
Objective function value is =  2250016.0


In [ ]:
#Printing the decision variables 
print('Decision Variables')
print('x1 = ', model_ex1.x1.value)  
print('x2 = ', model_ex1.x2.value)  

Decision Variables
x1 =  5.0
x2 =  10.0


Ex 1.3

**Constraints :** \\
model_ex1.constr_1=Constraint(expr=model.x1<=10) \\
model_ex1.constr_2=Constraint(expr=model.x1>=5) \\
model_ex1.constr_3=Constraint(expr=model.x2<=10) \\
model_ex1.constr_4=Constraint(expr=model.x2>=5)

In [ ]:
#New constraint addition in the model
model_ex1.constr_5 = Constraint(expr= ((model_ex1.x1)**2 +(model_ex1.x2)**2) - (14*model_ex1.x1 + 12*model_ex1.x2) <= -83)

In [ ]:
#Printing the details of model_ex1 that we have created
model_ex1.pprint()

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     5 :   5.0 :    10 : False : False :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     5 :  10.0 :    10 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : (1 - x1)**2 + ((x2 - x1**2)*100)**2

1 Constraint Declarations
    constr_5 : Size=1, Index=None, Active=True
        Key  : Lower : Body                            : Upper : Active
        None :  -Inf : x1**2 + x2**2 - (14*x1 + 12*x2) : -83.0 :   True

4 Declarations: x1 x2 obj constr_5


In [ ]:
opt_ipopt = SolverFactory('ipopt')
result_ex1 = opt_ipopt.solve(model_ex1)
print(result_ex1)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.02177906036376953
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [ ]:
print('The Solver status:', result_ex1.solver.status)
print('The Solver termination condition:',result_ex1.solver.termination_condition)
print('The Objective function value = ', model_ex1.obj())

The Solver status: ok
The Solver termination condition: optimal
The Objective function value =  6319145.95383758


In [ ]:
#Printing decision variables values
print('Decision Variables')
print('x1 = ', model_ex1.x1.value)  
print('x2 = ', model_ex1.x2.value)

Decision Variables
x1 =  5.59140679228246
x2 =  6.125960331590355


Ex 1.4

In [ ]:
#clear() method clears all references to local variables

In [ ]:
model_ex1.x1.clear()

In [ ]:
model_ex1.x2.clear()

In [ ]:
upper_bound = np.inf
model_ex1.x1.setub(upper_bound)
model_ex1.x2.setub(upper_bound)

In [ ]:
lower_bound = -np.inf
model_ex1.x1.setlb(lower_bound)
model_ex1.x2.setlb(lower_bound)

In [ ]:
#Printing the details of model_ex1 that we have created
model_ex1.pprint()

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  -inf :  None :   inf : False : False :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  -inf :  None :   inf : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : (1 - x1)**2 + ((x2 - x1**2)*100)**2

1 Constraint Declarations
    constr_5 : Size=1, Index=None, Active=True
        Key  : Lower : Body                            : Upper : Active
        None :  -Inf : x1**2 + x2**2 - (14*x1 + 12*x2) : -83.0 :   True

4 Declarations: x1 x2 obj constr_5


In [ ]:
opt_ipopt = SolverFactory('ipopt')
result_ex1 = opt_ipopt.solve(model_ex1)
print(result_ex1)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.044638872146606445
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [ ]:
print('The Solver status:', result_ex1.solver.status)
print('The Solver termination condition:',result_ex1.solver.termination_condition)
print('The Objective function = ', model_ex1.obj())

The Solver status: ok
The Solver termination condition: optimal
The Objective function =  6319145.95376243


In [ ]:
print('Decision Variables')
print('x1 = ', model_ex1.x1.value)  
print('x2 = ', model_ex1.x2.value)

Decision Variables
x1 =  5.591406792267957
x2 =  6.125960331577647
